In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import random
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.path import Path
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
import networkx as nx

In [2]:
P = 20
L = 20
d_max = 2
base_coor = [P/2, L/2, 0]

N = 200
m = 10
kc = 25
kd = 250
kt = kd + N
beta = 0.8
data_rate = 500 # bps
t_transmission =  ((kd * 8)/data_rate) + ((kc * 8)/data_rate) + (((kc + N) * 8)/data_rate)
initial_energy = 5
E_elec = 50 * 1e-9
E_amp =  10 * 1e-5
E_fs = 10*1e-12
E_mp = 0.0013*1e-12
p = 0.05


<h1>
    <b>Low Energy Adaptive Cluster Head (LEACH)</b>
</h1>
<p>
    LEACH merupakan protokol dari jaringan sensor nirkabel, yang menggunakan non-persistent CSMA pada fase Setup dan TDMA pada fase Steady State Phase.
</p>

<h3>
    <b>- Setup Phase</b>
</h3>

Ratio
$$ a = \frac{\tau_{\text{transmission}}}{\tau_{\text{propagation}}} $$

Throughput

$$ \rho = \frac{{k_c \cdot e^{-a \cdot k_c}}}{{k_c(1 + 2a) + e^{-a \cdot k_c}}} $$

Energy yang dibutuhkan untuk proses seleksi Cluster Head 

$$ E_{\text{selection}} = \frac{{k_c}}{{\rho}} \left( E_{\text{elec}} + E_{\text{amp}} \cdot d_{\text{CH−BS}}^2 \right) + \frac{{p \cdot N - 1}}{{\rho}} \left( \beta \cdot E_{\text{elec}} \cdot k_c \right) + E_{\text{elec}} \cdot k_t $$


Energy yang dibutuhkan untuk broadcasting status Cluster Head pada Node sekitarnya 

$$ E_{\text{adv-CH}} = k_c \cdot \left( E_{\text{elec}} + E_{\text{amp}} \cdot d_{\text{max}}^2 \right) $$


energy yang dibutuhkan untuk bergabung ke dalam cluster

$$ E_{\text{Join}} = p \cdot N \cdot k_c \cdot E_{\text{elec}} $$


<h3>
    <b>Steady State Phase</b>
</h3>

Energy yang dibutuhkan untuk melakukan broadcasting schedule pengiriman data menggunakan TDMA

$$ k_t = k_c + N $$


$$ E_{\text{TDMA-CH}} = N_c \cdot k_c \cdot E_{\text{elec}} + k_t \cdot (E_{\text{elec}} + E_{\text{amp}} \cdot d_{\text{max}}^2) $$


$$ E_{\text{TDMA-Node}} = \frac{k_c}{\rho} \cdot (E_{\text{elec}} + E_{\text{amp}} \cdot d_{\text{Node-CH}}^2) + \frac{N - 1}{\rho} \cdot \beta \cdot E_{\text{elec}} \cdot k_c + E_{\text{elec}} \cdot k_t $$


Energy yang dibutuhkan untuk mengirimkan data dari Cluster Head ke Base Station dan dari Node ke Cluster Head


$$ E_{\text{CH-BS}} = m \cdot \left[ (N_c \cdot k_d \cdot E_{\text{elec}}) + (N - N_c) \right] \cdot (\beta \cdot E_{\text{elec}} \cdot k_d) + k_d \cdot (E_{\text{elec}} + E_{\text{amp}} \cdot d_{\text{Node-CH}}^2) $$


$$ E_{\text{Node-CH}} = m \cdot k_d \cdot \left( E_{\text{elec}} + E_{\text{amp}} \cdot d_{\text{Node-CH}}^2 \right) $$


In [3]:
def ratio_delay(D, distance):
    v = 3 * 1e8
    t = distance/v
    a = t/t_transmission
    # print(f"Ratio delay : {a:.2f} | T_tranmission : {t_transmission:.2f} | T_uw : {t_uw:.2f}")
    return a

def throughput(a):
    numerator = kc * np.exp(-a * kc)
    denominator = kc * (1 + 2 * a) + np.exp(-a * kc)
    rho = numerator / denominator
    # print(f"Throughput : {rho:.2f} | numerator : {numerator:.2f} | denominator : {denominator:.2f}")
    return rho


class Node:
    def __init__(self, x, y, z, id):
        self.x = x
        self.y = y
        self.z = z
        self.energy = initial_energy
        self.nϵG = False
        self.alive = True
        self.CH = False
        self.which_cluster = 0
        self.eligible_round = 0
        self.cluster_class = 0
        self.id = id

    def distance(self, other_node):
        return np.sqrt((self.x - other_node.x)**2 + (self.y - other_node.y)**2)

    def reset(self):
        self.CH = False
        self.which_cluster = 0

    def advertisement(self, count_cluster, eligible_round):
        self.CH = True
        self.nϵG = False
        self.eligible_round = eligible_round
        self.which_cluster = count_cluster
    
    def energySelection(self, d_CH_BS, p):
        # If node selected as a CH
        a = ratio_delay(self.z, d_CH_BS)
        rho = throughput(a)
        first_term = (kc / rho) * (E_elec + E_amp * (d_CH_BS**2))
        second_term = (p * (N - 1) / rho) * (beta * E_elec * kc)
        third_term = E_elec * kt
        E_Selection = first_term + second_term + third_term
        return E_Selection

    def energyAdvertisement(self):
        # Broadcasting to all nodes in the range of d_max, occurs only for CH
        return kc * (E_elec + E_amp * (d_max**2))
    
    def energyJoin(self, p):
        # Node receive the broadcasting message and decide whether want to join as a associated node for CH i-th
        return p * N * kc * E_elec

    def energy_contention_TDMA_CH(self, Nc):
        return kc * Nc * E_elec + kt * (E_elec + E_amp * (d_max**2))
    
    def energy_contention_TDMA_Node(self, d_CH_Node):
        a = ratio_delay(self.z, d_CH_Node)
        rho = throughput(a)
        energy = (kc / rho) * (E_elec + E_amp * (d_CH_Node**2)) + ((N-1)/ rho) * kc * beta * E_elec + kt * E_elec
        return energy
    
    def energyFrame_CH(self, Nc, d_CH_BS):
        return m * Nc * kd * E_elec * beta * kd * E_elec + kd * (E_elec +  E_amp * (d_CH_BS**2))
    
    def energyFrame_Node(self, d_CH_Node):
        return m * kd * E_elec * (E_elec + E_elec +  E_amp * (d_CH_Node**2))
    

<h3>
    <b>Wireless Sensor Network Deployment</b>
</h3>

<p>
Pada paper tersebut, penulis menjelaskan bahwa sensor tersebut terdistribusi Poisson, dengan nilai lambda di dasarkan atas densitas sensor terhadap area.
</p>

$$ P(X = k) = \frac{e^{-\lambda} \lambda^k}{k!} $$


$$f(x) = \frac{1}{b - a}, \quad a \leq x \leq b $$


In [ ]:
def createNetworks():
    areaTotal = P * L

    #Point process parameters
    lambda0 = N/(P * L)                                             

    #Simulate a Poisson point process
    numbPoints = np.random.poisson(lambda0 * areaTotal)           
    xx = P * np.random.uniform(0,1,numbPoints)      
    yy = L * np.random.uniform(0,1,numbPoints)  

    nodes = []
    nodes.append(Node(P//2, L//2, 0, 0))

    for i in range(len(xx)):
        nodes.append(
            Node(xx[i], yy[i], 0, i + 1)
        )

    return nodes

<h3>
    <b>Low energy Adaptive Cluster Hierarchy (LEACH)</b>
</h3>
<p>
    Melakukan pemilihan Cluster Head berdasarkan nilai threshold dan ratio yang ditentukan
</p>

$$ T(n) = \frac{r}{1 - r \cdot (n \mod \frac{1}{r})} $$


<h3>
    <b>K-Means Clustering</b>
</h3>
<p>
    Melakukan pemilihan Cluster Head berdasarkan jarak node yang paling dekat dengan Centroid
</p>

$$ \mathbf{X}_i = \begin{bmatrix} x_i^1 & x_i^2  \end{bmatrix}^T $$

$$ J = \min_{\mu_j} \sum_{i=0}^{N-1} \sum_{j=0}^{K-1} r_{n,k} \lVert \mathbf{X}_i - \mu_j \rVert_2^2 $$


$$ r_{n,k} = \begin{cases} 1, & \text{if } k = \text{argmin}_j \left( \lVert \mathbf{X}_i - \boldsymbol{\mu}_j \rVert_2^2 \right) \\ 0, & \text{Otherwise} \end{cases} $$


$$ \frac{\partial}{\partial \mu_j} \sum_{i=0}^{N-1} \sum_{j=0}^{K-1} r_{n,k} \lVert \mathbf{X}_i - \mu_j \rVert_2^2 = 0 $$


$$ \mu_j = \frac{\sum_{i=0}^{N-1} r_{n,k} \mathbf{X}_i}{\sum_{i=0}^{N-1} r_{n,k}} $$



<p>
    Dengan nilai k-optimum di dapatkan dari paper [3], yakni :
</p>

$$ k_{\text{opt}} = \sqrt{\frac{N}{2\pi}} \times \frac{\sqrt{\epsilon_{\text{fs}}}}{\sqrt{\epsilon_{\text{mp}}}} \times \frac{M}{{d_{\text{CH-BS}}^2}} $$

<h3>
    <b>DEKCS: A Dynamic Clustering Protocol to
Prolong Underwater Sensor Networks</b>
</h3>


<p>
    Melakukan pemilihan Cluster Head berdasarkan antar node terkecil
</p>





$$ J = \min_{\mu_j} \sum_{i=0}^{N-1} \sum_{j=0}^{K-1} r_{n,k} \lVert \mathbf{X}_i - \mu_j \rVert_2^2 $$


<h3>
    <b>Metode yang digunakan: Multi Agent Reinforcement Learning + DECKS</b>
</h3>

<p>
    Menggunakan DECKS untuk pemilihan cluster head dan menggunakan RL untuk proses routing pengiriman data
</p>

$$ Q_{(t+1)}^i(s_t, a_t) = Q_t^i(s_t, a_t) + \eta \left[ r(s_t, a_t) + \gamma \max_{a_{t}} \left\{ Q_t^i(s_{(t+1)}, a) - Q_t^i(s_t, a_t) \right\} \right] $$


In [ ]:
class multiHop(object):
    def __init__(self,graph):
        self.graph = graph
        self.adjacent_mat = nx.adjacency_matrix(graph).todense()
        self.num_nodes = len(self.adjacent_mat)
        self.adjacent_mat = nx.adjacency_matrix(graph, nodelist=range(self.num_nodes)).toarray()#:D
        # print(f"\n Adjacent Matrix \n{self.adjacent_mat}\n")

    def q_learning(self,start_state=0, aim_state = 10, num_epoch=200, gamma=0.8, epsilon=0.05, alpha=0.1):

        len_of_paths = []
        rewards = self.rewardMapping(aim_state)
        # print(f"\n Reward \n{rewards}\n")
        
    
        q_table = np.zeros((self.num_nodes, self.num_nodes))  # num_states * num_actions
        for episode in range(1, num_epoch + 1):
            #print(f"========================================================================== Episode : {episode} =========================================================================")
            current_state = start_state
            path = [current_state]
            len_of_path = 0
            while True:
                next_state = self.epsilon_greedy(current_state, q_table, start_state, epsilon=epsilon)
                s_next_next = self.epsilon_greedy(next_state, q_table, start_state, epsilon=-0.2)  # epsilon<0, greedy policy
                # update q_table
                reward = rewards[current_state][next_state]
                delta = reward + gamma * q_table[next_state, s_next_next] - q_table[current_state, next_state]
                
                q_table[current_state, next_state] = q_table[current_state, next_state] + alpha * delta
                # update current state
                current_state = next_state
                len_of_path += -reward
                path.append(current_state)
                # print(f"==========================\n{q_table}==========================\n")
                # print(f"reward: {reward} | Current state : {current_state} | Next state : {next_state} \n\n")


                if current_state == aim_state:
                    break
            len_of_paths.append(len_of_path)

            
        # print(f"Q learning table : \n{q_table}")
        return path

    def epsilon_greedy(self,s_curr, q, start_state, epsilon):#exploraiton vs exploitation 
        try :
            potential_next_states = np.where(np.array(self.adjacent_mat[s_curr]) > 0)[0]
        except IndexError as e:
            print(e)
            print(f"{self.adjacent_mat[s_curr]}")
        potential_next_states = potential_next_states[potential_next_states != start_state]
        # print(f"potential next state : {potential_next_states}")
        if random.random() > epsilon:  
            q_of_next_states = q[s_curr][potential_next_states]
            s_next = potential_next_states[np.argmax(q_of_next_states)]
            # print(f"q_of_next_states : {q_of_next_states}   |   s_next : {s_next}")
        else:  
            s_next = random.choice(potential_next_states)
        return s_next
    
    def rewardMapping(self, aim_state):
        r = self.adjacent_mat
        r[:, aim_state] = 100
        r[aim_state, :] = 100
        for i in range(0, len(r)):
            for j in range(0, len(r)):
                if i == j :
                    r[i][j] = -5
        return r


class networkEnvironment:
    def __init__(self, nodes, mode):
        self.nodes = nodes
        self.mode = mode
        self.alive_data = []
        self.energy_data = []
        self.centroids = [[], []]
    
    def showResult(self, hop):
        fig, ax = plt.subplots(1,2, figsize=(20,5))
        rounds = np.array([i for i in range(0, len(self.alive_data))])
        self.alive_data = np.array(self.alive_data)
        self.energy_data = np.array(self.energy_data)

        ax[0].plot(rounds, self.alive_data, color='k')
        ax[0].scatter(rounds[::hop], self.alive_data[::hop], marker='o', edgecolor='k', color='r')
        ax[0].set_ylabel("Node Alive")
        ax[0].set_xlabel("Round")

        ax[1].plot(rounds, self.energy_data, color='k')
        ax[1].scatter(rounds[::hop], self.energy_data[::hop], marker='o', edgecolor='k', color='r')
        ax[1].set_ylabel("Energy Consumed")
        ax[1].set_xlabel("Round")

        ax[1].yaxis.set_major_locator(MultipleLocator(np.max(self.energy_data)/10))
        ax[1].yaxis.set_major_formatter('{x:.0f}')
        ax[1].yaxis.set_minor_locator(MultipleLocator(np.max(self.energy_data)/20))
        ax[1].xaxis.set_major_locator(MultipleLocator(np.max(rounds)/10))
        ax[1].xaxis.set_major_formatter('{x:.0f}')
        ax[1].xaxis.set_minor_locator(MultipleLocator(np.max(rounds)/20))

        ax[0].yaxis.set_major_locator(MultipleLocator(np.max(self.alive_data)/10))
        ax[0].yaxis.set_major_formatter('{x:.0f}')
        ax[0].yaxis.set_minor_locator(MultipleLocator(np.max(self.alive_data)/20))
        ax[0].xaxis.set_major_locator(MultipleLocator(np.max(rounds)/10))
        ax[0].xaxis.set_major_formatter('{x:.0f}')
        ax[0].xaxis.set_minor_locator(MultipleLocator(np.max(rounds)/20))

    def showNetwork(self, simulation_round):
        fig, ax = plt.subplots()
        nodes_x = [node.x for node in self.nodes if ((node.id != 0) and (node.CH == False) and (node.which_cluster != 0) and (node.energy > 0))]
        nodes_y = [node.y for node in self.nodes if ((node.id != 0) and (node.CH == False) and (node.which_cluster != 0) and (node.energy > 0))] 

        nodes_orphan_x = [node.x for node in self.nodes if ((node.id != 0) and (node.CH == False) and (node.which_cluster == 0) and (node.energy > 0))]
        nodes_orphan_y = [node.y for node in self.nodes if ((node.id != 0) and (node.CH == False) and (node.which_cluster == 0) and (node.energy > 0))] 

        CH_x = [node.x for node in self.nodes if ((node.id != 0) and (node.CH == True))]
        CH_y = [node.y for node in self.nodes if ((node.id != 0) and (node.CH == True))]
        BS_x, BS_y = [node.x for node in self.nodes if node.id == 0], [node.y for node in self.nodes if node.id == 0]
        
        dead_nodes_x = [node.x for node in self.nodes if ((node.id != 0) and (node.energy < 0))]
        dead_nodes_y = [node.y for node in self.nodes if ((node.id != 0) and (node.energy < 0))] 

        ax.scatter(nodes_x, nodes_y                 , marker="o",color='purple', edgecolors='k', label = "Node")
        ax.scatter(nodes_orphan_x, nodes_orphan_y   , marker="o",color='c', edgecolors='k', label = "Orphan Node")
        ax.scatter(CH_x, CH_y                       , marker="o",color="g", edgecolors='k', label = "Cluster Head")
        ax.scatter(dead_nodes_x, dead_nodes_y       , marker="x",color='r', label="Dead Node")
        ax.scatter(BS_x, BS_y                       , marker="s",color="b", edgecolors='k', label="Base station")
        ax.scatter(self.centroids[0], self.centroids[1]                       , marker="s",color="b", edgecolors='k', label="Base station")

        font = {
                'color':  'black',
                'weight': 'bold'
                }
        
        ax.set_title(f'Round : {simulation_round}', fontdict=font)
        ax.set_xlabel('Length (m)')
        ax.set_ylabel('Width (m)')
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
              fancybox=True, shadow=True, ncol=5, markerscale=1, fontsize=10)
        plt.tight_layout()
        plt.show()

    def is_alive_and_eligible(self, node):
        status = ((node.energy > 0) and (node.alive == True) and (node.eligible_round == 0))
        return status

    def is_lessEqual_than_threshold(self, round_number):
        if random.uniform(0, 1) <= p/(1-p * (round_number % (1/p))):
            return True
        else:
            return False
    
    def euclidean_distance(self, nodeA, nodeB):
        return np.sqrt(np.sum((nodeA - nodeB)**2))

    def kmeans(self, X, k, iteration=100):
        # Memilih random k centroid sebagai nilai awal
        centroids = X[np.random.choice(X.shape[0], k)]

        for i in range(iteration):
            # Euclidian Distance
            distances = np.linalg.norm(X[:, None] - centroids, axis=2)
            labels = np.argmin(distances, axis=1)
            # Mengupdate nilai Centroid
            new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])
            centroids = new_centroids
        
        return centroids, labels
    

    def CHselection(self, simulation_round):
        # Memilih Cluster Head
        print(f"Round : {simulation_round}")
        for node in self.nodes:
            node.reset()
            
        if self.mode == "LEACH":
            for node in self.nodes:
                if ((self.is_lessEqual_than_threshold(simulation_round)) and (self.is_alive_and_eligible(node)) and (node.id != 0)):
                    node.CH = True
                    node.which_cluster = node.id
                    node.eligible_round = 1/p
        
        elif self.mode == "K-Means":
            X, id = [], []
            for node in self.nodes:
                if ((node.id != 0) and (node.alive) and (node.energy > 0)) :
                    X.append([node.x, node.y])
                    id.append(node.id)
                    
            X = np.array(X)
            elbow = []
            max_k = 30  # Maximum number of clusters to try
            for k in range(1, max_k + 1):
                centroids, labels = self.kmeans(X, k)
                error = np.sum((X - centroids[labels])**2)
                elbow.append(error)
        
            # Calculate the change in distortions and find the elbow point
            elbow = np.array(elbow)
            elbow_diff = np.diff(elbow, prepend=elbow[0])
            acceleration = np.diff(elbow_diff, prepend=elbow_diff[0])
            optimal_k = np.argmax(acceleration)   
            centroids, labels = self.kmeans(X, optimal_k)
            print(f"K-opt : {optimal_k}")
            ch_id = []
            for ch in centroids:
                distances = [[], []]
                for node in self.nodes:
                    if node.id != 0:
                        X = np.array([node.x, node.y])
                        CH = np.array([ch[0], ch[1]])
                        distances[0].append(self.euclidean_distance(X, CH))
                        distances[1].append(node.id)
                    
                ch_id.append(distances[1][np.argmin(distances[0])])

            for node in self.nodes:
                if node.id != 0:
                    for id in ch_id:
                        if node.id == id:
                            node.CH = True
                            node.which_cluster = id

        elif self.mode == "DECKS":
            X, id = [], []
            for node in self.nodes:
                if ((node.id != 0) and (node.alive) and (node.energy > 0)) :
                    X.append([node.x, node.y])
                    id.append(node.id)
                    
            X = np.array(X)
            elbow = []
            max_k = 30  # Maximum number of clusters to try
            for k in range(1, max_k + 1):
                centroids, labels = self.kmeans(X, k)
                error = np.sum((X - centroids[labels])**2)
                elbow.append(error)
        
            # Calculate the change in distortions and find the elbow point
            elbow = np.array(elbow)
            elbow_diff = np.diff(elbow, prepend=elbow[0])
            acceleration = np.diff(elbow_diff, prepend=elbow_diff[0])
            optimal_k = np.argmax(acceleration)   
            self.centroids, labels = self.kmeans(X, optimal_k)
            print(f"K-opt : {optimal_k}")
            ch_id = []
            for ch in self.centroids:
                distances = [[], []]
                for node in self.nodes:
                    if node.id != 0:
                        X = np.array([node.x, node.y])
                        CH = np.array([ch[0], ch[1]])
                        distances[0].append(self.euclidean_distance(X, CH))
                        distances[1].append(node.id)
                    
                ch_id.append(distances[1][np.argmin(distances[0])])

            for node in self.nodes:
                if node.id != 0:
                    for id in ch_id:
                        if node.id == id:
                            node.CH = True
                            node.which_cluster = id


   
            
        # Node Bergabung ke Cluster Head
        CHs = [node for node in self.nodes if node.CH]  
        count = 0
        if len(CHs) != 0: 
            for node in self.nodes:
                if node.id != 0:
                    distances = [[], []]
                    for ch in CHs:
                        X = np.array([node.x, node.y])
                        CH = np.array([ch.x, ch.y])
                        distances[0].append(self.euclidean_distance(X, CH))
                        distances[1].append(ch.which_cluster)
                    
                    if distances[0][np.argmin(distances[0])] > d_max:
                        # Jika jarak ke Cluster Head lebih besar dari Jarak Jangkauan Transmisi, maka 
                        node.which_cluster = 0
                        count += 1
                    else:
                        node.which_cluster = distances[1][np.argmin(distances[0])]   
        else:
            # Jika tidak ada yang terpilih sebagai CH, maka node akan diam
            for node in self.nodes:
                if node.id != 0:
                    node.which_cluster = 0
        print(f"Idle Node : {count}")
        return len(CHs)
    
    def Nc(self, which_cluster):
        count = 0
        for node in self.nodes:
            if node.which_cluster == which_cluster:
                count += 1
        return count

    def SetupPhase(self, simulation_round):
        self.k = self.CHselection(simulation_round)
        energy_total = sum([node.energy for node in self.nodes])
        print(f"Setup phase energy initial : {energy_total}")
        if self.k != 0:
            for node in self.nodes:
                if node.id != 0:
                    if node.CH:
                        d_CH_BS = self.euclidean_distance(np.array([node.x, node.y, node.z]), np.array(base_coor))
                        energy_dissipated = node.energyAdvertisement() + node.energySelection(d_CH_BS, self.k/N)
                        node.energy = node.energy - energy_dissipated
                    else:
                        node.energy = node.energy - node.energyJoin(k/N)
        else:
            print(f"There is no transmission in {simulation_round} round")

        energy_total = sum([node.energy for node in self.nodes])
        print(f"Setup phase energy after : {energy_total}")

    def SteadyStatePhase(self):
        BS = [node for node in self.nodes if node.id == 0][0]
        CHs = [node for node in self.nodes if node.CH]

        if len(CHs) != 0:
            energy_total = sum([node.energy for node in self.nodes])
            print(f"Contention phase energy initial : {energy_total:.2f}")
            for node in self.nodes:
                if node.id != 0 :
                    if node.CH == True:
                        Nc = self.Nc(node.which_cluster)
                        node.energy = node.energy - node.energy_contention_TDMA_CH(Nc)
                    else:
                        for CH in CHs:
                            if node.which_cluster == CH.which_cluster : 
                                d_CH_Node = self.euclidean_distance(np.array([node.x, node.y, node.z]), np.array([CH.x, CH.y, CH.z]))
                                node.energy = node.energy - node.energy_contention_TDMA_Node(d_CH_Node)
            energy_total = sum([node.energy for node in self.nodes])

            print(f"Transmission energy begin : {energy_total:.2f}")
            if self.mode == "LEACH":
                for node in self.nodes:
                    if node.id != 0 :
                        if node.CH == True:
                            Nc = self.Nc(node.which_cluster)
                            d_CH_BS = self.euclidean_distance(np.array([node.x, node.y]), np.array([BS.x, BS.y]))
                            node.energy = node.energy - node.energyFrame_CH(Nc, d_CH_BS)
                        else:
                            for CH in CHs:
                                if node.which_cluster == CH.which_cluster :
                                    d_CH_Node = self.euclidean_distance(np.array([node.x, node.y]), np.array([CH.x, CH.y]))
                                    node.energy = node.energy - node.energyFrame_Node(d_CH_Node)
        for node in self.nodes:    
            node.eligible_round -= 1
            if node.eligible_round < 0:
                node.eligible_round = 0
            if node.energy < 0:
                node.alive = False

        energy_total = sum([node.energy for node in self.nodes])
        node_alive = len([node.energy for node in self.nodes if node.alive])
        print(f"Node Alive : {node_alive}")

        self.alive_data.append(node_alive)
        self.energy_data.append(energy_total)

    def startSimulation(self, rounds):
        self.showNetwork(0)
        for simulation_round in range(1, rounds):
            self.SetupPhase(simulation_round)
            self.SteadyStatePhase()
            if simulation_round % 20 == 0:
                self.showNetwork(simulation_round)
        self.showResult(10)

        return self.alive_data, self.energy_data


node_LEACH = createNetworks()
node_K_Means = createNetworks()
node_DECKS = createNetworks()
node_proposed = createNetworks()

In [ ]:
a = np.array([1,1,2,4,5])
np.unique(a)

In [ ]:
LEACH = networkEnvironment(node_LEACH, "LEACH")
LEACH_aliveNode, LEACH_EnergyNode = LEACH.startSimulation(200)

In [ ]:
K_Means = networkEnvironment(node_K_Means, "K-Means")
K_Means_aliveNode, LEACH_EnergyNode = K_Means.startSimulation(200)

<h3>
    <b>REFERENCES</b>
</h3>

<ol>
    <li>Omeke, Kenechi G., et al. "DEKCS: A dynamic clustering protocol to prolong underwater sensor networks." IEEE Sensors Journal 21.7 (2021): 9457-9464.</li>
    <li>Lazarou, Georgios Y., Jing Li, and Joseph Picone. "A cluster-based power-efficient MAC scheme for event-driven sensing applications." Ad Hoc Networks 5.7 (2007): 1017-1030.</li>
    <li>Heinzelman, Wendi B., Anantha P. Chandrakasan, and Hari Balakrishnan. "An application-specific protocol architecture for wireless microsensor networks." IEEE Transactions on wireless communications 1.4 (2002): 660-670.</li>
</ol>